# Examples for a basic query against OEDI for ResStock

## Import modules

In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd

## Reload changes
If you are a developer and are making changes to the code, signal to reload changes each time a block is executed

In [2]:
# auto reload
%load_ext autoreload
%autoreload 2


## Initialize BuildStock Query Object
Connect to the database, setup the tables, set the source schema, and set project workgroup

In [3]:
my_run = BuildStockQuery(
    'rescore', # Change to your project working group
    'buildstock_sdr',
    (
        "resstock_2024_amy2018_release_2_metadata_state_vu", # Baseline metadata view
        "resstock_2024_amy2018_release_2_by_state_vu", # Timeseries view
        "resstock_2024_amy2018_release_2_metadata_state_vu" # Upgrade metadata view (same as Baseline for OEDI)
    ),
    db_schema="resstock_oedi",
    skip_reports=True
)

INFO:buildstock_query.query_core:Loading ('resstock_2024_amy2018_release_2_metadata_state_vu', 'resstock_2024_amy2018_release_2_by_state_vu', 'resstock_2024_amy2018_release_2_metadata_state_vu') ...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso


## Simple annual query

In [4]:
baseline_agg = my_run.agg.aggregate_annual(
    enduses=['electricity.total.energy_consumption'],
    group_by=['geometry_building_type_recs']
)
baseline_agg

INFO:botocore.tokens:Loading cached SSO token for nrel-sso


,geometry_building_type_recs,sample_count,units_count,electricity.total.energy_consumption
0,Mobile Home,34492,8.702388e+06,9.781267e+10
1,Multi-Family with 2 - 4 Units,43991,1.109900e+07,8.643677e+10
2,Multi-Family with 5+ Units,101069,2.549987e+07,1.807385e+11
3,Single-Family Attached,32134,8.107461e+06,7.740386e+10
4,Single-Family Detached,338032,8.528603e+07,1.168146e+12


## Simple timeseries query

In [5]:
ts_agg = my_run.agg.aggregate_timeseries(
    enduses=['out.electricity.total.energy_consumption'],
    restrict=[('state', ['CO'])],
    upgrade_id=0,
    timestamp_grouping_func='month',
    group_by=['geometry_building_type_recs', 'state'],
    get_query_only=False
)
print(ts_agg)

INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.


      geometry_building_type_recs state  timestamp  sample_count  \
0                     Mobile Home    CO 2018-01-01           391   
1                     Mobile Home    CO 2018-02-01           391   
2                     Mobile Home    CO 2018-03-01           391   
3                     Mobile Home    CO 2018-04-01           391   
4                     Mobile Home    CO 2018-05-01           391   
5                     Mobile Home    CO 2018-06-01           391   
6                     Mobile Home    CO 2018-07-01           391   
7                     Mobile Home    CO 2018-08-01           391   
8                     Mobile Home    CO 2018-09-01           391   
9                     Mobile Home    CO 2018-10-01           391   
10                    Mobile Home    CO 2018-11-01           391   
11                    Mobile Home    CO 2018-12-01           391   
12  Multi-Family with 2 - 4 Units    CO 2018-01-01           469   
13  Multi-Family with 2 - 4 Units    CO 2018-02-

## Utility unit count query

In [6]:
units = my_run.utility.aggregate_unit_counts_by_eiaid(
    eiaid_list=['4110', '14328'], # ComEd and PG&E,
    get_query_only=False
)
print(units)
units.to_csv("utility_units.csv")

INFO:buildstock_query.utility_query:Aggregating unit counts by eiaid


   eiaid  sample_count   units_count
0  14328         28730  1.828840e+09
1   4110         15880  1.010859e+09


## Utility annual query

In [7]:
enduses = [
    'out.site_energy.net.energy_consumption',
    'out.site_energy.total.energy_consumption'
]

# By default just does all the eiaids
ts_utility_agg = my_run.utility.aggregate_annual_by_eiaid(
    enduses=enduses,
    get_query_only=False
)
print(ts_utility_agg)
ts_utility_agg.to_csv("utility_annual_agg.csv")

      eiaid  sample_count   units_count  site_energy.net.energy_consumption  \
0     10000          4227  2.690744e+08                        9.389554e+12   
1     10005          1801  1.146447e+08                        3.853915e+12   
2     10009          3206  2.040815e+08                        3.782175e+12   
3     10012           319  2.030630e+07                        6.944541e+11   
4     10019           630  4.010335e+07                        1.411753e+12   
...     ...           ...           ...                                 ...   
1754   9964           658  4.188572e+07                        1.232310e+12   
1755    998           108  6.874861e+06                        2.305716e+11   
1756   9991           385  2.450761e+07                        1.057302e+12   
1757   9996           278  1.769640e+07                        5.840864e+11   
1758   9999          1976  1.257845e+08                        4.618860e+12   

      site_energy.total.energy_consumption  
0     

## Utility timeseries query

In [8]:
enduses = [
    'out.site_energy.net.energy_consumption',
    'out.site_energy.total.energy_consumption'
]
group_by=['timestamp'] # If no timestamp, then annual values will be returned

ts_utility_agg = my_run.utility.aggregate_ts_by_eiaid(
    eiaid_list=['4110', '14328'], # ComEd and PG&E
    enduses=enduses,
    group_by=group_by,
    get_query_only=False
)
print(ts_utility_agg)
ts_utility_agg.to_csv("utility_ts_agg.csv")

INFO:buildstock_query.utility_query:Will submit request for ['4110', '14328']
INFO:buildstock_query.utility_query:Submitting query for ['4110', '14328']
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.
INFO:buildstock_query.query_core:{'submitted': 0, 'running': 0, 'pending': 1, 'completed': 0, 'failed': 0}
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.query_core:{'submitted': 0, 'running': 0, 'pending': 1, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:Submitted queries[0] (5a9041e3-6a79-45cc-ad9d-571ebbb78084)
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 1, 'pending': 0, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 1, 'pending': 0, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 1, 'pending': 0, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_cor

       eiaid           timestamp  sample_count   units_count  \
0      14328 2018-01-01 00:15:00         28730  3.417140e+06   
1      14328 2018-01-01 00:30:00         28730  3.417140e+06   
2      14328 2018-01-01 00:45:00         28730  3.417140e+06   
3      14328 2018-01-01 01:00:00         28730  3.417140e+06   
4      14328 2018-01-01 01:15:00         28730  3.417140e+06   
...      ...                 ...           ...           ...   
70075   4110 2018-12-31 23:00:00         15880  2.563967e+06   
70076   4110 2018-12-31 23:15:00         15880  2.563967e+06   
70077   4110 2018-12-31 23:30:00         15880  2.563967e+06   
70078   4110 2018-12-31 23:45:00         15880  2.563967e+06   
70079   4110 2019-01-01 00:00:00         15880  2.563967e+06   

       site_energy.net.energy_consumption  \
0                            2.730670e+06   
1                            2.796730e+06   
2                            2.832369e+06   
3                            2.909646e+06   
4     